# AKS Cookbook

## 🧪 Service Bus Queue scaled by KEDA

![visual](visual.png)

This setup will go through creating an Azure Service Bus queue and deploying this consumer with the ScaledObject to scale via KEDA. If you already have an Azure Service Bus namespace you can use your existing queues.

KEDA works alongside standard Kubernetes components like the Horizontal Pod Autoscaler and can extend functionality without overwriting or duplication.

Keda got two key roles within the cluster, **keda-operator** which scales from minimum to maximum pod counts set in the ScaledObject manifest file via Kubernetes Horizontal Pod Autoscaler and **keda-operator-metrics-apiserver** which gets the data for the scaling decision.

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Set your Azure Subscription](#1)
- [2️⃣ Get AKS credentials and verify the connection](#2)
- [3️⃣ Update your AKS cluster to enable KEDA](#3)
- [4️⃣ Create a Service Bus instance](#4)
- [5️⃣ Create a new authorization rule](#5)
- [6️⃣ Deploy order processor](#6)
- [7️⃣ Deploying our autoscaling](#7)
- [8️⃣ Publishing messages to the queue](#8)
- [🗑️ Clean up resources](#clean)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Provide either the tenant and the subscription IDs, before running the lab.
- Provide also a resource group name.
- Adjust the location parameters according your preferences and on the [product availability by Azure region](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management).
- Check your aks cluster name. If you want to create a new AKS cluster, please refer to this [Jupyter notebook](new-aks-cluster.ipynb)

In [ ]:
import os
import datetime

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"
tenant_id = "<tenant_id>"
subscription_id = "<subscription_id>"
aks_resource_name = "aks-keda-service-bus"
aks_resource_namespace = "keda-service-bus-sample"
service_bus_namespace = "service-bus-orders"
service_bus_sku = "basic"
service_bus_queue = "orders"
service_bus_auth_rule = "order-consumer"

<a id='1'></a>
### 1️⃣ Set your Azure Subscription
Use this step to specify which Azure tenant and subscription should be used for subsequent Azure CLI commands.

In [ ]:
! az login --tenant {tenant_id}
! az account set --subscription {subscription_id}

<a id='2'></a>
### 2️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())


<a id='3'></a>
### 3️⃣ Update your AKS cluster to enable KEDA

We'll enable KEDA add-on on an existing cluster. If you want to create a new AKS cluster, please refer to this [Jupyter notebook](new-aks-cluster.ipynb)

In [ ]:
# Get the credentials for the AKS cluster.
! az aks get-credentials --resource-group {resource_group_name} --name {aks_resource_name}

! az aks update --resource-group {resource_group_name} --name {aks_resource_name} --enable-keda 

# Verify the KEDA add-on is installed on your cluster.
keda_enabled_output = ! az aks show --resource-group {resource_group_name} --name {aks_resource_name} --query "workloadAutoScalerProfile.keda.enabled" -o tsv
keda_enabled = keda_enabled_output[-1].strip()
if keda_enabled == 'true':
    print("✅ KEDA add-on is enabled on the AKS cluster ⌚ ", datetime.datetime.now().time())
else:
    print("❌ KEDA add-on is not enabled on the AKS cluster ⌚ ", datetime.datetime.now().time())

# Verify KEDA is running on the cluster.
keda_pods = ! kubectl get pods -n kube-system
print("keda pods: ", keda_pods)

# Verify KEDA version.
keda_version = ! kubectl get crd/scaledobjects.keda.sh -o yaml
print("keda version: ", keda_version)

Verify KEDA is running on the cluster.

In [ ]:
! kubectl get pods -n kube-system

Verify KEDA version

In [ ]:
! kubectl get crd/scaledobjects.keda.sh -o yaml

<a id='4'></a>
### 4️⃣ Create a Service Bus instance
Skip this step if you already have a sb instance and with a queue.

In [ ]:
import os
import datetime
import json

def log(stdout, name, action):
    if stdout.n.startswith("ERROR"):
        print("👎🏻 ", name, " was NOT ", action, ": ", stdout)
    else:
        print("👍🏻 ", name, " was ", action, " ⌚ ", datetime.datetime.now().time())

def parse_output(output):
    try:
        return json.loads("".join(output))
    except json.JSONDecodeError:
        return None

# Create a new Azure Service Bus namespace
sb_namespace_stdout = ! az servicebus namespace create --name {service_bus_namespace} --resource-group {resource_group_name} --sku {service_bus_sku}
log(sb_namespace_stdout, "Azure Service Bus namespace", "created")

# Parse the namespace creation output to check if it was successful
sb_namespace = parse_output(sb_namespace_stdout)
if sb_namespace and sb_namespace.get("provisioningState") == "Succeeded":
    print(f"✅ Azure Service Bus namespace {service_bus_namespace} created successfully ⌚ {datetime.datetime.now().time()}")
else:
    print(f"❌ Failed to create Azure Service Bus namespace {service_bus_namespace} ⌚ {datetime.datetime.now().time()}")

# Create a queue in the Service Bus namespace
sb_queue_stdout = ! az servicebus queue create --name {service_bus_queue} --namespace-name {service_bus_namespace} --resource-group {resource_group_name}
log(sb_queue_stdout, "Azure Service Bus queue '{service_bus_queue}'", "created")
  
# Parse the queue creation output to check if it was successful  
sb_queue = parse_output(sb_queue_stdout)
if sb_queue and sb_queue.get("status") == "Active":
    print(f"✅ Azure Service Bus queue '{service_bus_queue}' created successfully ⌚ {datetime.datetime.now().time()}")
else:
    print(f"❌ Failed to create Azure Service Bus queue '{service_bus_queue}' ⌚ {datetime.datetime.now().time()}")

<a id='5'></a>
### 5️⃣ Create a new authorization rule
We create a new authorization rule, called 'order-consumer' with Manage Send Listen permissions which our .net app will use to process messages.

In [ ]:
! az servicebus queue authorization-rule create --resource-group {resource_group_name} --namespace-name {service_bus_namespace} --queue-name {service_bus_queue} --name order-consumer --rights Manage Listen Send

Once the authorization rule is created, we can list the connection string as following:

In [ ]:
import os  
import datetime  
import json  
import base64  
  
def log(stdout, name, action):
    if stdout[0].startswith("ERROR"):
        print(f"👎🏻 {name} was NOT {action}: {stdout}")
    else:
        print(f"👍🏻 {name} was {action} ⌚ {datetime.datetime.now().time()}")

def parse_output(output):
    try:
        return json.loads("".join(output))
    except json.JSONDecodeError:
        return None

auth_rule_stdout = ! az servicebus queue authorization-rule create --resource-group {resource_group_name} --namespace-name {service_bus_namespace} --queue-name {service_bus_queue} --name order-consumer --rights Manage Listen Send
log(auth_rule_stdout, "Authorization rule 'order-consumer'", "created")

# List the keys for the authorization rule
keys_stdout = ! az servicebus queue authorization-rule keys list --resource-group {resource_group_name} --namespace-name {service_bus_namespace} --queue-name {service_bus_queue} --name order-consumer -o json
keys = parse_output(keys_stdout)
if keys:
    primary_connection_string = keys.get("primaryConnectionString")
    print("Primary Connection String:", primary_connection_string) 
else:
    print("❌ Failed to retrieve keys for the authorization rule 'order-consumer' ⌚", datetime.datetime.now().time())

# Create the base64 representation of the connection string
if primary_connection_string:
    primary_connection_string_base64 = base64.b64encode(primary_connection_string.encode("utf-8")).decode("utf-8")
    print("Base64 Encoded Connection String:", primary_connection_string_base64)
else:
    primary_connection_string_base64 = ""
    print("No primary connection string found to encode.")

<a id='6'></a>
### 6️⃣ Deploy order processor

We will start by creating a new Kubernetes namespace to run our order processor in. If you want to create a new AKS cluster, please refer to this [Jupyter notebook](new-aks-cluster.ipynb) 

In [ ]:
! kubectl create namespace {aks_resource_namespace}

Before we can connect to our queue, we need to create a secret which contains the Service Bus connection string to the queue.

In [ ]:
import os
import subprocess

# Read the secret template
with open("deploy-app.yaml", "r") as file:
    deployapp_template = file.read()

# Replace placeholders with actual values
deployapp_content = deployapp_template.replace("$SERVICE_BUS_QUEUE_NAME", service_bus_queue).replace("$BASE64_ENCODED_CONNECTION_STRING", primary_connection_string_base64)

# Write the updated content to a new file
with open("deploy-app_modified.yaml", "w") as file:
    file.write(deployapp_content)

# Apply the secret using kubectl
apply_command = "kubectl apply -f deploy-app_modified.yaml --namespace " + aks_resource_namespace
result = subprocess.run(apply_command, shell = True, capture_output = True, text = True)

# Print the result
if result.returncode == 0:
    print("Command applied successfully.")
    print(result.stdout)
else:
    print("Failed.")
    print(result.stderr)

Once created, you should be able to retrieve the secret

In [ ]:
! kubectl get secrets -n {aks_resource_namespace}

You will see that our deployment shows up with one pods created.

In [ ]:
! kubectl get deployments --namespace {aks_resource_namespace} -o wide

<a id='7'></a>
### 7️⃣ Deploying our autoscaling

We will create a new authorization rule, 'keda-monitor', with Management permissions so that KEDA can monitor it.

In [ ]:
! az servicebus queue authorization-rule create --resource-group {resource_group_name} --namespace-name {service_bus_namespace} --queue-name {service_bus_queue} --name keda-monitor --rights Manage Send Listen

In [ ]:
import os
import subprocess

# Read the secret template
with open("deploy-autoscaling.yaml", "r") as file:
    deployapp_template = file.read()

# Replace placeholders with actual values
deployapp_content = deployapp_template.replace("$BASE64_ENCODED_CONNECTION_STRING", primary_connection_string_base64).replace("$SERVICE_BUS_QUEUE_NAME", service_bus_queue)

# Write the updated content to a new file
with open("deploy-autoscaling_modified.yaml", "w") as file:
    file.write(deployapp_content)

# Apply the secret using kubectl
apply_command = "kubectl apply -f deploy-autoscaling_modified.yaml --namespace "+ aks_resource_namespace
result = subprocess.run(apply_command, shell = True, capture_output = True, text = True)

# Print the result
if result.returncode == 0:
    print("Command applied successfully.")
    print(result.stdout)
else:
    print("Failed.")
    print(result.stderr)

Once created, you will see that our deployment shows up with no pods created.
This is because our queue is empty and KEDA scaled it down until there is work to do.

In [ ]:
! kubectl get deployments --namespace {aks_resource_namespace} -o wide

<a id='8'></a>
### 8️⃣ Publishing messages to the queue

The following job will send messages to the "orders" queue on which the order processor is listening to. As the queue builds up, KEDA will help the horizontal pod autoscaler add more and more pods until the queue is drained. The order generator will allow you to specify how many messages you want to queue.

Clone the project

In [ ]:
! git clone https://github.com/kedacore/sample-dotnet-worker-servicebus-queue

Run the project

Go to __sample-dotnet-worker-servicebus-queue\src\Keda.Samples.Dotnet.OrderGenerator\Program.cs__ and replace
- QueueName
- ConnectionString

with the correct values. Then run the **OrderGenerator** project.

In [ ]:
! dotnet run --project "./sample-dotnet-worker-servicebus-queue/src/Keda.Samples.Dotnet.OrderGenerator\Keda.Samples.Dotnet.OrderGenerator.csproj"

Now that the messages are generated, you'll see that KEDA starts automatically scaling out your deployment.

In [ ]:
! kubectl get deployments --namespace {aks_resource_namespace} -o wide
! kubectl get pods --namespace {aks_resource_namespace}

You can look at the logs for a given processor as following.

In [ ]:
! kubectl logs -l app=order-processor --namespace {aks_resource_namespace}